In [ ]:
import numpy as np
import plotly.graph_objects as go
import pymesh
import torch
import torch.nn as nn
import torch.nn.functional as F

from siren import Siren

%load_ext autoreload
%autoreload 2

Load terrain mesh from file

In [ ]:
mesh = pymesh.load_mesh("data/3dmodels/landscape_example.obj")

In [ ]:
# Plot point cloud
fig = go.Figure(data=[go.Scatter3d(
    x=mesh.vertices[:, 0],
    y=mesh.vertices[:, 1],
    z=mesh.vertices[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=mesh.vertices[:, 2],                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)])
fig.update_layout(width=1600, height=900)
fig.update_layout(scene = dict(aspectmode='data'))
fig.show()

In [ ]:
# Plot mesh
fig = go.Figure(data=[go.Mesh3d(x=mesh.vertices[:,0],
                                y=mesh.vertices[:,1],
                                z=mesh.vertices[:,2],
                                i=mesh.faces[:,0],
                                j=mesh.faces[:,1],
                                k=mesh.faces[:,2],
                                color='lightpink',
                                opacity=0.50)])
# Set figsize
fig.update_layout(width=1600, height=900)
fig.update_layout(scene = dict(aspectmode='data'))
fig.show()

Plan a path over the mesh
 - Assume we're given some global plan
 - Now, use a local planner which evaluates the heightmap to determine a local costmap

In [ ]:
# Generate traversability costmap for terrain


Neural elevation map

In [ ]:
# Simpler simulated map
X, Y = torch.meshgrid(torch.linspace(-1, 1, 100), torch.linspace(-1, 1, 100))
X = X.reshape(-1, 1)
Y = Y.reshape(-1, 1)
z = 0.1 * (torch.sin(2 * X) + torch.cos(2 * Y))
xy = torch.cat((X, Y), dim=1)

In [ ]:
# plot 
fig = go.Figure(data=[go.Scatter3d(
    x=X.reshape(-1),
    y=Y.reshape(-1),
    z=z.reshape(-1),
    mode='markers',
    marker=dict(
        size=2,
        color=z.reshape(-1),                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)])
fig.update_layout(width=1200, height=700)
fig.update_layout(scene = dict(aspectmode='data'))
fig.show()

Full mesh               

In [ ]:
# X = mesh.vertices[:,0]
# Y = mesh.vertices[:,1]
# Z = mesh.vertices[:,2]

# x = torch.tensor(X, dtype=torch.float32)
# y = torch.tensor(Y, dtype=torch.float32)
# z = torch.tensor(Z, dtype=torch.float32)

# xy = torch.stack([x, y], dim=1)

Partial mesh

In [ ]:
bound = 50
mask = np.logical_and(mesh.vertices[:,0] <= bound, mesh.vertices[:,0] >= bound)
X = mesh.vertices[mask,0]
Y = mesh.vertices[mask,1]
Z = mesh.vertices[mask,2]

x = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(Y, dtype=torch.float32)
z = torch.tensor(Z, dtype=torch.float32)

xy = torch.stack([x, y], dim=1)
z = z[:,None]

In [ ]:
from collections import OrderedDict

class DNN(torch.nn.Module):
    """Deep neural network.
    
    """
    def __init__(self, layers):
        """Initialize the network.
        
        Parameters
        ----------
        layers : list of int
            List of layer dimensions.

        """
        super(DNN, self).__init__()
        
        # parameters
        self.depth = len(layers) - 1
        
        # set up layer order dict
        #self.activation = torch.nn.ReLU
        self.activation = torch.nn.Tanh
        #self.activation = Sine
        
        layer_list = list()
        for i in range(self.depth - 1): 
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation()))
            
        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        layerDict = OrderedDict(layer_list)
        
        # deploy layers
        self.layers = torch.nn.Sequential(layerDict)
        
    def forward(self, x):
        """ Forward pass. """
        out = self.layers(x)
        return out

In [ ]:
# Neural net to predict z from (x,y)
# TODO: try different activation functions
#  - tanh
#  - sine
class Net(nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = nn.Linear(n_feature, n_hidden)   # hidden layer
        self.predict = nn.Linear(n_hidden, n_output)   # output layer
        self.activation = F.relu

    def forward(self, x):
        x = self.activation(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x

In [ ]:
#net = Net(n_feature=2, n_hidden=10, n_output=1)
#net = DNN([2, 100, 100, 1])
net = Siren(in_features=2, out_features=1, hidden_features=256, hidden_layers=3, outermost_linear=True)

#optimizer = torch.optim.SGD(net.parameters(), lr=0.05)
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)
loss_fn = torch.nn.MSELoss()  # this is for regression mean squared loss

In [ ]:
# # Siren weight initialization
# # For each layer, draw weights from U(-sqrt(6/n), sqrt(6/n)) where n is input dimension to layer

# for layer in net.layers:
#     if isinstance(layer, nn.Linear):
#         w = layer.weight.data
#         n = w.shape[1]
#         layer.weight.data = (2 * torch.rand(w.shape) - 1) * torch.sqrt(torch.tensor(6 / n))

In [ ]:
n_epochs = 1000

for epoch in range(n_epochs):
    # Forward pass: Compute predicted y by passing x to the model
    
    z_pred, coords = net(xy)

    # Compute and print loss
    loss = loss_fn(z_pred, z)
    if epoch % 100 == 0:
        print('epoch: ', epoch,' loss: ', loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
# Plot NN prediction
fig = go.Figure(data=[go.Scatter3d(
    x=X.reshape(-1),
    y=Y.reshape(-1),
    z=z_pred.detach().numpy().reshape(-1),
    mode='markers',
    marker=dict(
        size=2,
        color=z_pred.detach().numpy().reshape(-1),                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)])
fig.update_layout(width=1200, height=700)
fig.update_layout(scene = dict(aspectmode='data'))
fig.show()


Plan a path through the neural surface

In [ ]:
# Go from (-1, -1) to (1, 1)
# cost(x,y) = 